In [9]:
import numpy as np
import random
import time
from IPython.display import display,Latex,Math
%matplotlib inline
 
from IPython.core.interactiveshell import InteractiveShell
sh = InteractiveShell.instance()
 
def number_to_str(n,cut=5):
    ns=str(n)
    format_='{0:.'+str(cut)+'f}'
    if 'e' in ns or ('.' in ns and len(ns)>cut+1):
        return format_.format(n)
    else:
        if n == 0:
            return ("{\color{blue} %s}" %n)
        return str(n)
 
def matrix_to_latex(mat,style='bmatrix'):
    if type(mat)==np.matrixlib.defmatrix.matrix:
        mat=mat.A
    head=r'\begin{'+style+'}'
    tail=r'\end{'+style+'}'
    if len(mat.shape)==1:
        body=r'\\'.join([str(el) for el in mat])
        return head+body+tail
    elif len(mat.shape)==2:
        lines=[]
        for row in mat:
            lines.append('&'.join([number_to_str(el)  for el in row])+r'\\')
        s=head+' '.join(lines)+tail
        return s
    return None
 
sh.display_formatter.formatters['text/latex'].type_printers[np.ndarray]=matrix_to_latex

In [10]:
N_KIND = 3 * 10

def generate_map():
    sample_list = random.choices(range(1, N_KIND), k=32) * 2
    return np.array(sample_list, dtype=np.int).reshape(8, 8)

main_map = generate_map()
main_map = np.pad(main_map,((1,1),(1,1)),'constant',constant_values = (0,0)) 
print(main_map)

[[ 0  0  0  0  0  0  0  0  0  0]
 [ 0 13 24 22 14 20  6 20  9  0]
 [ 0  9  9 13 22 26 16 10 14  0]
 [ 0  8 15  3  9  9 29 21 22  0]
 [ 0 27 22 16 16 27 17  3  5  0]
 [ 0 13 24 22 14 20  6 20  9  0]
 [ 0  9  9 13 22 26 16 10 14  0]
 [ 0  8 15  3  9  9 29 21 22  0]
 [ 0 27 22 16 16 27 17  3  5  0]
 [ 0  0  0  0  0  0  0  0  0  0]]


In [11]:

def link(x1, y1, x2, y2):
    if main_map[x1, y1] == main_map[x2, y2]:
        print("Linked (%s, %s) to (%s, %s)." % (x1, y1, x2, y2))
        main_map[x1, y1] = 0
        main_map[x2, y2] = 0
    else:
        print('error!')

def direct(x1, y1, x2, y2):
    if x1 == x2 and y1 != y2:    # 平行y轴
        return not main_map[x1, y1:y2:(2*(y1 < y2)-1)][1:].any()
    if y1 == y2 and x1 != x2:    # 平行x轴
        return not main_map[x1:x2:(2*(x1 < x2)-1), y1][1:].any()
    return False

def one_corner(x1, y1, x2, y2):
    p1 = direct(x1, y2, x1, y1) and direct(x1, y2, x2, y2) and not main_map[x1, y2] # x1, y2
    p2 = direct(x2, y1, x1, y1) and direct(x2, y1, x2, y2) and not main_map[x2, y1] # x2, y1
    return p1 or p2
    
def two_corner(x1, y1, x2, y2):
    for x in range(10):  # (x, y1) (x, y2)
        if not(main_map[x, y1] or main_map[x, y2])\
        and direct(x1, y1, x, y1) and direct(x, y1, x, y2) and direct(x, y2, x2, y2):
            return True

    for y in range(10):  # (x1, y) (x2, y)
        if not(main_map[x1, y] or main_map[x2, y])\
        and direct(x1, y1, x1, y) and direct(x1, y, x2, y) and direct(x2, y, x2, y2):
            return True
    
    return False

def three_corner(x1, y1, x2, y2):
    for x in range(10):
        for y in range(10):
            p1 = not (main_map[x1, y] or main_map[x, y] or main_map[x, y2]) and\
                direct(x1, y1, x1, y) and direct(x1, y, x, y) and direct(x, y, x, y2) and direct(x, y2, x2, y2)
            p2 = not (main_map[x, y1] or main_map[x, y] or main_map[x2, y]) and\
                direct(x1, y1, x, y1) and direct(x, y1, x, y) and direct(x, y, x2, y) and direct(x2, y, x2, y2)
            if p1 or p2:
                return True
    return False

def get_score(x1, y1, x2, y2):
    if direct(x1, y1, x2, y2):
#         print('1 line.')
        return 50
    elif one_corner(x1, y1, x2, y2):
#         print('2 lines.')
        return 10
    elif two_corner(x1, y1, x2, y2):
#         print('3 lines.')
        return 20
    elif three_corner(x1, y1, x2, y2):
#         print('4 lines.')
        return 0
    else:
#         print('more than 4 lines.')
        return None

In [12]:
main_map

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 13, 24, 22, 14, 20,  6, 20,  9,  0],
       [ 0,  9,  9, 13, 22, 26, 16, 10, 14,  0],
       [ 0,  8, 15,  3,  9,  9, 29, 21, 22,  0],
       [ 0, 27, 22, 16, 16, 27, 17,  3,  5,  0],
       [ 0, 13, 24, 22, 14, 20,  6, 20,  9,  0],
       [ 0,  9,  9, 13, 22, 26, 16, 10, 14,  0],
       [ 0,  8, 15,  3,  9,  9, 29, 21, 22,  0],
       [ 0, 27, 22, 16, 16, 27, 17,  3,  5,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [22]:
import sys
sys.path.append("../../auto_grader/")
sys.path.append("../../read_picture/")
sys.path.append("../../mnist_data/")
import link_search
import auto_grader

In [23]:
ag = auto_grader.auto_grader(False)


FileNotFoundError: [Errno 2] No such file or directory: '../mnist_data/t10k-images.idx3-ubyte'

In [ ]:
ls = link_search.link_search()